<a href="https://colab.research.google.com/github/SKUMARI2015/MLOps-Jan2025/blob/main/Copy_of_G24AIT020_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Step 1: Install Required Libraries
!pip install torch wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
#Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import wandb
import numpy as np

In [ ]:

#Define the Model Architecture
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self._to_linear = None
        self.calculate_flattened_size()
        self.fc1 = nn.Linear(self._to_linear, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 10)

    def calculate_flattened_size(self):
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, 28, 28)
            x = torch.relu(self.conv1(dummy_input))
            x = nn.MaxPool2d(2, 2)(x)
            x = self.dropout1(x)
            x = torch.relu(self.conv2(x))
            x = nn.MaxPool2d(2, 2)(x)
            x = self.dropout2(x)
            self._to_linear = x.view(1, -1).size(1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = nn.MaxPool2d(2, 2)(x)
        x = self.dropout1(x)
        x = torch.relu(self.conv2(x))
        x = nn.MaxPool2d(2, 2)(x)
        x = self.dropout2(x)
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define model parameters
input_size = 784  # Example: MNIST dataset
hidden_size = 128
output_size = 10  # Example: 10 classes for MNIST
model = SimpleMLP(input_size, hidden_size, output_size)

In [ ]:
#Step 4: Set Up WandB and Configuration
# Initialize WandB
wandb.init(project="MLOps2025_G24AIT020", config={
    "learning_rate": 0.001,
    "batch_size": 32,
    "epochs": 5,
    "architecture": "Two Fully Connected Layers"
})

config = wandb.config

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sarita-gps2015 (sarita-gps2015-prom-iitj) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
#Q1: Data Preprocessing and Train-Validation Split
# Load MNIST dataset
from torchvision import datasets, transforms

# Preprocessing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Train-Validation Split (80:20)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.22MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 154kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.45MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.34MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
#Step 6: Initialize Model, Loss, and Optimizer
#model = SimpleMLP(config.input_size, config.hidden_size, config.output_size)
model = SimpleMLP(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)


In [ ]:
#Step 7: Training and Validation
#Q3: Training and Validation
# Loss and Optimizer
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

# Training and Validation Loop
def train_model(model, train_loader, val_loader, criterion, optimizer):
    for epoch in range(config.epochs):
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0

        for inputs, labels in train_loader:
            #inputs = inputs.view(inputs.size(0), -1)  # Flatten the input
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_accuracy = 100 * train_correct / train_total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                #inputs = inputs.view(inputs.size(0), -1)  # Flatten the input
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total

        # Log metrics to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss / len(train_loader),
            "train_accuracy": train_accuracy,
            "val_loss": val_loss / len(val_loader),
            "val_accuracy": val_accuracy
        })

        print(f"Epoch [{epoch + 1}/{config.epochs}], "
            f"Train Loss: {train_loss / len(train_loader):.4f}, "
            f"Train Accuracy: {train_accuracy:.2f}%, "
            f"Val Loss: {val_loss / len(val_loader):.4f}, "
            f"Val Accuracy: {val_accuracy:.2f}%")
    wandb.finish()

train_model(model, train_loader, val_loader, criterion, optimizer)

Epoch [1/5], Train Loss: 0.2279, Train Accuracy: 92.85%, Val Loss: 0.0779, Val Accuracy: 97.64%
Epoch [2/5], Train Loss: 0.0856, Train Accuracy: 97.37%, Val Loss: 0.0563, Val Accuracy: 98.38%
Epoch [3/5], Train Loss: 0.0666, Train Accuracy: 98.00%, Val Loss: 0.0493, Val Accuracy: 98.50%
Epoch [4/5], Train Loss: 0.0551, Train Accuracy: 98.28%, Val Loss: 0.0468, Val Accuracy: 98.53%
Epoch [5/5], Train Loss: 0.0475, Train Accuracy: 98.46%, Val Loss: 0.0577, Val Accuracy: 98.25%


epoch,▁▃▅▆█
train_accuracy,▁▇▇██
train_loss,█▂▂▁▁
val_accuracy,▁▇██▆
val_loss,█▃▂▁▃
epoch,5
train_accuracy,98.4625
train_loss,0.04753
val_accuracy,98.25
val_loss,0.05768


In [ ]:
sweep_config = {
    "method": "grid",
    "parameters": {
        "learning_rate": {
            "values": [0.001, 0.01, 0.1]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="MLOps2025_G24AIT020")

def sweep_train():

        with wandb.init() as run:
             config = run.config
             #model = A3NN(hidden_size=config.hidden_size).to(device)
             model = SimpleMLP(input_size=728, hidden_size=128, output_size=10)
             criterion = nn.CrossEntropyLoss()
             optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
             train_model(model, train_loader, val_loader, criterion, optimizer)
             wandb.finish()
# Run sweep
wandb.agent(sweep_id, function=sweep_train)

Create sweep with ID: 1ke188ca
Sweep URL: https://wandb.ai/sarita-gps2015-prom-iitj/MLOps2025_G24AIT020/sweeps/1ke188ca


wandb: Agent Starting Run: 5ddfw0i7 with config:
wandb: 	learning_rate: 0.001


Epoch [1/5], Train Loss: 0.2371, Train Accuracy: 92.76%, Val Loss: 0.0907, Val Accuracy: 97.11%
Epoch [2/5], Train Loss: 0.0886, Train Accuracy: 97.24%, Val Loss: 0.0584, Val Accuracy: 98.18%
Epoch [3/5], Train Loss: 0.0693, Train Accuracy: 97.83%, Val Loss: 0.0477, Val Accuracy: 98.62%
Epoch [4/5], Train Loss: 0.0552, Train Accuracy: 98.32%, Val Loss: 0.0454, Val Accuracy: 98.75%
Epoch [5/5], Train Loss: 0.0475, Train Accuracy: 98.49%, Val Loss: 0.0431, Val Accuracy: 98.76%


epoch,▁▃▅▆█
train_accuracy,▁▆▇██
train_loss,█▃▂▁▁
val_accuracy,▁▆▇██
val_loss,█▃▂▁▁
epoch,5
train_accuracy,98.4875
train_loss,0.04753
val_accuracy,98.75833
val_loss,0.04306


wandb: Agent Starting Run: ejrzzuls with config:
wandb: 	learning_rate: 0.01


...

epoch,▁▃▅▆█
train_accuracy,▁▆▇██
train_loss,█▂▂▁▁
val_accuracy,▁▇▅██
val_loss,█▁▅▂▂
epoch,5
train_accuracy,93.29167
train_loss,0.25495
val_accuracy,97.025
val_loss,0.11594


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: z6zcfm3n with config:
wandb: 	learning_rate: 0.1


Epoch [1/5], Train Loss: 3.4189, Train Accuracy: 10.21%, Val Loss: 2.3111, Val Accuracy: 10.73%
Epoch [2/5], Train Loss: 2.4471, Train Accuracy: 10.64%, Val Loss: 2.3109, Val Accuracy: 11.08%
Epoch [3/5], Train Loss: 2.3561, Train Accuracy: 10.35%, Val Loss: 2.3199, Val Accuracy: 10.73%
Epoch [4/5], Train Loss: 2.3145, Train Accuracy: 10.22%, Val Loss: 2.3096, Val Accuracy: 10.73%
Epoch [5/5], Train Loss: 2.3146, Train Accuracy: 10.37%, Val Loss: 2.3202, Val Accuracy: 10.73%


epoch,▁▃▅▆█
train_accuracy,▁█▃▁▄
train_loss,█▂▁▁▁
val_accuracy,▁█▁▁▁
val_loss,▂▂█▁█
epoch,5
train_accuracy,10.36875
train_loss,2.3146
val_accuracy,10.73333
val_loss,2.32017


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
# Save the trained model
import torch
torch.save(model.state_dict(), "model.pth")

# Log model as artifact
artifact = wandb.Artifact("trained_model", type="model")
artifact.add_file("model.pth")
wandb.init()
wandb.log_artifact(artifact)

<Artifact trained_model>